<a href="https://colab.research.google.com/github/shruti-madhav/Sentiment_Analysis/blob/main/Sentimnt_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install neattext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 1.9 MB/s eta 0:00:00


In [ ]:
# import neattext as nt
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import neattext.functions as nfs

In [ ]:
# Dataset from kaggle
data = pd.read_csv('/content/IMDB Dataset.csv')
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.info

In [ ]:
data.describe()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
sns.countplot(x='sentiment',data=data)

VADER - old model
      - bag of words approach
      
RoBERTa - Pretrained model
        - from hugging face

NLTK lib used